---
# PREPROCESS RAW DATA

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import gc

#needed to import utils.py
sys.path.append('../') 

import utils
import utils_preprocessing

import numpy as np
import pandas as pd

%matplotlib inline  

---
# 1. Read data from the `demographics` csv file

In [2]:
#set the data directory
data_dir = os.path.abspath('../01_raw_data')

#set the name of CSV file
data_file = f'{data_dir}/PROACT_DEMOGRAPHICS.csv'

#read data and show some info
df = utils.read_csv(data_file)

print(df.shape[0])

df.head()

11675


,subject_id,Demographics_Delta,Age,Date_of_Birth,Ethnicity,Race_Americ_Indian_Alaska_Native,Race_Asian,Race_Black_African_American,Race_Hawaiian_Pacific_Islander,Race_Unknown,Race_Caucasian,Race_Other,Race_Other_Specify,Sex
0,89,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male
1,329,0.0,38.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,Female
2,348,0.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Female
3,406,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male
4,411,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male


## Show columns stats and missingness

In [3]:
utils.show_columns_stats(df)

subject_id...................... = 11675 rows (100.0%)     0 with NaN (  0.0%) Uniques= 11675 
Demographics_Delta.............. = 11590 rows (99.27%)    85 with NaN ( 0.73%) Uniques=    12 
Age............................. =  8654 rows (74.12%)  3021 with NaN (25.88%) Uniques=   195 
Date_of_Birth................... =  1298 rows (11.12%) 10377 with NaN (88.88%) Uniques=  1235 
Ethnicity....................... =  2936 rows (25.15%)  8739 with NaN (74.85%) Uniques=     4 
Race_Americ_Indian_Alaska_Native =    12 rows (  0.1%) 11663 with NaN ( 99.9%) Uniques=     2 
Race_Asian...................... =    73 rows ( 0.63%) 11602 with NaN (99.37%) Uniques=     2 
Race_Black_African_American..... =   136 rows ( 1.16%) 11539 with NaN (98.84%) Uniques=     2 
Race_Hawaiian_Pacific_Islander.. =     1 rows ( 0.01%) 11674 with NaN (99.99%) Uniques=     2 
Race_Unknown.................... =    25 rows ( 0.21%) 11650 with NaN (99.79%) Uniques=     2 
Race_Caucasian.................. =  7581 rows (64.

## Calculate the Age for those samples having `Age=NaN` and `Date_of_Birth<>NaN`
### Calculation based on delta days in Date_of_Birth column

In [4]:
#get samples with Age=NaN and Date_of_Birth<>NaN
df_calc_age_from_birth = df.loc[(df.Age.isnull()) & (df.Date_of_Birth.isnull()==False)]
print(f'Age calculated for {utils.get_quantity_of_rows(df_calc_age_from_birth)} samples')

#calculate the Age from the Date_of_Birth delta (in days and negative)
ages_calculated = df_calc_age_from_birth.Date_of_Birth.apply( lambda x: utils.calculate_age_from_birth_delta(np.abs(x)) )

#update samples with the calculated Age
df.loc[df_calc_age_from_birth.index,'Age'] = ages_calculated



Age calculated for 944 samples


##### We can note that now the Age column has 82.21% of non-NaN values (previously had 74.12%) 

In [5]:
utils.show_columns_stats(df, ['Age'])

Age =  9598 rows (82.21%)  2077 with NaN (17.79%) Uniques=   195 


### Remove irrelevant columns for this study, which are:
- `Demographics_Delta` column
- `Date_of_Birth` column (not necessary, using `Age` column), 
- `Ethnicity` column (high number of missing values)
- All `Race_*` columns (high number of missing values)

In [6]:
irrelevant_cols = ['Demographics_Delta', 'Date_of_Birth', 'Ethnicity', 'Race_Americ_Indian_Alaska_Native', 'Race_Asian',
                   'Race_Black_African_American', 'Race_Hawaiian_Pacific_Islander', 'Race_Unknown', 'Race_Caucasian', 
                   'Race_Other', 'Race_Other_Specify']

df.drop(columns=irrelevant_cols, inplace=True)
df.head()

,subject_id,Age,Sex
0,89,NaN,Male
1,329,38.0,Female
2,348,52.0,Female
3,406,NaN,Male
4,411,NaN,Male


----
----
----
## Remove rows with NaN in the AGE and SEX columns

In [7]:
# AGE column
to_delete = df.loc[(df.Age.isnull())]
df = utils.remove_rows(df=df, to_delete=to_delete, info='AGE')


# SEX column
to_delete = df.loc[(df.Sex.isnull())]
df = utils.remove_rows(df=df, to_delete=to_delete, info='SEX')

df


  - AGE Previous=11675, To delete=2077, After=9598
  - SEX Previous=9598, To delete=0, After=9598


,subject_id,Age,Sex
1,329,38.0,Female
2,348,52.0,Female
5,533,65.0,Female
6,586,63.0,Male
7,649,48.0,Female
...,...,...,...
11670,22564,63.0,Male
11671,160135,67.0,Female
11672,304192,59.0,Female
11673,820199,49.0,Male


### Check if all columns contain values (i.e., without missing values)

In [8]:
utils.show_columns_stats(df, df.columns)

subject_id =  9598 rows (100.0%)     0 with NaN (  0.0%) Uniques=  9598 
Age....... =  9598 rows (100.0%)     0 with NaN (  0.0%) Uniques=   194 
Sex....... =  9598 rows (100.0%)     0 with NaN (  0.0%) Uniques=     2 


----
## Save the pre-processed data to CSV file

In [9]:
dir_dest = os.path.abspath('../03_preprocessed_data/')
csv_dest = f'{dir_dest}/patient.csv'
utils.save_to_csv(df=df, csv_file=csv_dest)


9598 samples were saved


---
# 2. Preprocessing `ALS History`
---

## Read ALS History data file

In [10]:
#set the data directory
data_dir = os.path.abspath('../01_raw_data/')

#set the name of CSV file
data_file = f'{data_dir}/PROACT_ALSHISTORY.csv'

#read data and show some info
df_raw = utils.read_csv(data_file)

df_raw.head()

,subject_id,Site_of_Onset___Bulbar,Site_of_Onset___Limb,Site_of_Onset___Limb_and_Bulbar,Site_of_Onset___Other,Site_of_Onset___Other_Specify,Site_of_Onset___Spine,Subject_ALS_History_Delta,Disease_Duration,Symptom,Symptom_Other_Specify,Location,Location_Other_Specify,Site_of_Onset,Onset_Delta,Diagnosis_Delta
0,89,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,329,NaN,1.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-1706.0,-1068.0
2,348,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Onset: Other,-501.0,-302.0
3,406,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,411,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Show columns stats and missingness

In [11]:
utils.show_columns_stats(df_raw) 


subject_id..................... = 12936 rows (100.0%)     0 with NaN (  0.0%) Uniques= 10271 
Site_of_Onset___Bulbar......... =  1281 rows (  9.9%) 11655 with NaN ( 90.1%) Uniques=     3 
Site_of_Onset___Limb........... =  3589 rows (27.74%)  9347 with NaN (72.26%) Uniques=     3 
Site_of_Onset___Limb_and_Bulbar =     0 rows (  0.0%) 12936 with NaN (100.0%) Uniques=     1 
Site_of_Onset___Other.......... =     0 rows (  0.0%) 12936 with NaN (100.0%) Uniques=     1 
Site_of_Onset___Other_Specify.. =     0 rows (  0.0%) 12936 with NaN (100.0%) Uniques=     1 
Site_of_Onset___Spine.......... =     0 rows (  0.0%) 12936 with NaN (100.0%) Uniques=     1 
Subject_ALS_History_Delta...... =  9583 rows (74.08%)  3353 with NaN (25.92%) Uniques=     5 
Disease_Duration............... =     0 rows (  0.0%) 12936 with NaN (100.0%) Uniques=     1 
Symptom........................ =  2656 rows (20.53%) 10280 with NaN (79.47%) Uniques=    11 
Symptom_Other_Specify.......... =    50 rows ( 0.39%) 12886 

## Correct the duplicate `subject_id` problem in `RAW Data`


NOTE: apparently the reason for duplication `subject_id` values is because in #1 row was assigned the `Symptom` column, and the #2 row the `Onset_Delta` column

SOLUTION: group samples by `subject_id` column, and use `first()` function to get the first `non-NaN` value for each column

After merge values, 10,271 samples remained with no duplicated `subject_id`

In [12]:
# Correct the duplicate subject_id problem in RAW Data
df_raw = df_raw.groupby(['subject_id']).first().reset_index()
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10271 entries, 0 to 10270
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   subject_id                       10271 non-null  int64  
 1   Site_of_Onset___Bulbar           1281 non-null   float64
 2   Site_of_Onset___Limb             3589 non-null   float64
 3   Site_of_Onset___Limb_and_Bulbar  0 non-null      float64
 4   Site_of_Onset___Other            0 non-null      float64
 5   Site_of_Onset___Other_Specify    0 non-null      float64
 6   Site_of_Onset___Spine            0 non-null      float64
 7   Subject_ALS_History_Delta        6918 non-null   float64
 8   Disease_Duration                 0 non-null      float64
 9   Symptom                          2221 non-null   object 
 10  Symptom_Other_Specify            48 non-null     object 
 11  Location                         2818 non-null   object 
 12  Location_Other_Spe

## Preprocess ALS History

In [13]:
df_als_history = utils_preprocessing.preprocess_als_history(df=df_raw)
df_als_history.head()


,subject_id,Symptoms_Onset_Delta,Diagnosis_Delta,Site_Onset
0,89,NaN,NaN,Limb/Spinal
1,329,-1706.0,-1068.0,Limb/Spinal
2,348,-501.0,-302.0,Other
3,406,NaN,NaN,Limb/Spinal
4,411,NaN,NaN,Bulbar


## Join $Patient$ and $ALS\ History$ dataFrames

In [14]:
df = utils.join_datasets_by_key(df_main=df, df_to_join=df_als_history, key_name='subject_id', how='left')

df.head()

,subject_id,Age,Sex,Symptoms_Onset_Delta,Diagnosis_Delta,Site_Onset
1,329,38.0,Female,-1706.0,-1068.0,Limb/Spinal
2,348,52.0,Female,-501.0,-302.0,Other
5,533,65.0,Female,-1023.0,-44.0,Bulbar
6,586,63.0,Male,-715.0,-507.0,Bulbar
7,649,48.0,Female,-341.0,NaN,Bulbar


---
# 3. Preprocessing `Diagnosis_Delay`
---

## Calculate Diagnosis_Delay in months and codify it
```
 - Short   : <=  8 months   
 - Average :  >  8 and <= 18 months  
 - Long    :  > 18 months     
```

In [17]:
df = utils_preprocessing.preprocess_diagnosis_delay(df_to_process=df)

df.head()

,subject_id,Age,Sex,Symptoms_Onset_Delta,Diagnosis_Delta,Site_Onset,Diagnosis_Delay
1,329,38.0,Female,-1706.0,-1068.0,Limb/Spinal,Long
2,348,52.0,Female,-501.0,-302.0,Other,Short
5,533,65.0,Female,-1023.0,-44.0,Bulbar,Long
6,586,63.0,Male,-715.0,-507.0,Bulbar,Short
7,649,48.0,Female,-341.0,NaN,Bulbar,NaN


---
# 4. Preprocessing `Age_at_Onset`
---

## Calculate ages at onset months and codify them into ranges:
```
 - 0-39
 - 40-49
 - 50-59
 - 60-69
 - 70+
```

In [22]:
df = utils_preprocessing.preprocess_age_at_onset(df_to_process=df)

df.head()

,subject_id,Age,Sex,Symptoms_Onset_Delta,Diagnosis_Delta,Site_Onset,Diagnosis_Delay,Age_at_Onset
1,329,38.0,Female,-1706.0,-1068.0,Limb/Spinal,Long,0-39
2,348,52.0,Female,-501.0,-302.0,Other,Short,50-59
5,533,65.0,Female,-1023.0,-44.0,Bulbar,Long,60-69
6,586,63.0,Male,-715.0,-507.0,Bulbar,Short,60-69
7,649,48.0,Female,-341.0,NaN,Bulbar,NaN,40-49


---
# 5. Delete samples having NaN values in the following columns:
 - `Age_at_Onset`
 - `Diagnosis_Delay`
 - `Site_Onset`
---

In [24]:
# Age_of_Onset
to_delete = df.loc[(df.Age_at_Onset.isnull())]
df = utils.remove_rows(df=df, to_delete=to_delete)

# Diagnosis_Delay
to_delete = df.loc[(df.Diagnosis_Delay.isnull())]
df = utils.remove_rows(df=df, to_delete=to_delete)

# Site_Onset
to_delete = df.loc[(df.Site_Onset.isnull())]
df = utils.remove_rows(df=df, to_delete=to_delete)

  -  Previous=9598, To delete=1845, After=7753
  -  Previous=7753, To delete=2430, After=5323
  -  Previous=5323, To delete=52, After=5271


---
# 6. Save the pre-processed data to CSV file
---

In [26]:
dir_dest = os.path.abspath('../03_preprocessed_data/')
csv_dest = f'{dir_dest}/patient.csv'
utils.save_to_csv(df=df, csv_file=csv_dest)


5271 samples were saved


---
# 7. Preprocessing Last_Visit for each patient (in months from symptoms onset)
---

In [33]:
df = utils_preprocessing.preprocess_last_visit(df_to_process=df, data_dir=data_dir)

df.head()

 - Get Last_Visit registered in PROACT_ALSFRS
 - Get Last_Visit registered in PROACT_FVC
 - Get Last_Visit registered in PROACT_DEATHDATA
 - Get Last_Visit registered in PROACT_LABS
 - Get Last_Visit registered in PROACT_RILUZOLE
 - Get Last_Visit registered in PROACT_SVC
 - Get Last_Visit registered in PROACT_VITALSIGNS
 - Get Last_Visit registered in PROACT_ALSHISTORY
 - Get Last_Visit registered in PROACT_DEMOGRAPHICS
 - Get Last_Visit registered in PROACT_ELESCORIAL
 - Get Last_Visit registered in PROACT_FAMILYHISTORY
 - Get Last_Visit registered in PROACT_HANDGRIPSTRENGTH
 - Get Last_Visit registered in PROACT_MUSCLESTRENGTH
 - Get Last_Visit registered in PROACT_TREATMENT
 - Get Last_Visit registered in PROACT_ADVERSEEVENTS
 - Get Last_Visit registered in PROACT_ADVERSEEVENTS
 - Get Last_Visit registered in PROACT_CONMEDS
 - Get Last_Visit registered in PROACT_CONMEDS


,subject_id,Age,Sex,Symptoms_Onset_Delta,Diagnosis_Delta,Site_Onset,Diagnosis_Delay,Age_at_Onset,Last_Visit_Delta,Last_Visit_from_Onset
1,329,38.0,Female,-1706.0,-1068.0,Limb/Spinal,Long,0-39,287.0,65
2,348,52.0,Female,-501.0,-302.0,Other,Short,50-59,401.0,29
5,533,65.0,Female,-1023.0,-44.0,Bulbar,Long,60-69,404.0,46
6,586,63.0,Male,-715.0,-507.0,Bulbar,Short,60-69,3.0,23
8,708,46.0,Male,-1070.0,-986.0,Bulbar,Short,40-49,250.0,43


---
# 8. Preprocessing Death data
---

In [52]:
df = utils_preprocessing.preprocess_death_data(df_to_process=df, data_dir=data_dir)

df.head(15)

,subject_id,Age,Sex,Symptoms_Onset_Delta,Diagnosis_Delta,Site_Onset,Diagnosis_Delay,Age_at_Onset,Event_Dead,Event_Dead_Time_from_Onset
1,329,38.0,Female,-1706.0,-1068.0,Limb/Spinal,Long,0-39,False,65.0
2,348,52.0,Female,-501.0,-302.0,Other,Short,50-59,False,29.0
5,533,65.0,Female,-1023.0,-44.0,Bulbar,Long,60-69,False,46.0
6,586,63.0,Male,-715.0,-507.0,Bulbar,Short,60-69,False,23.0
8,708,46.0,Male,-1070.0,-986.0,Bulbar,Short,40-49,True,43.0
9,902,76.8,Female,-514.0,-49.0,Limb/Spinal,Average,70+,False,21.0
11,1064,66.0,Female,-550.0,-462.0,Bulbar,Short,60-69,False,32.0
13,1309,58.0,Male,-638.0,-169.0,Limb/Spinal,Average,50-59,False,21.0
14,1312,67.0,Female,-247.0,-63.0,Bulbar,Short,60-69,False,8.0
17,1736,40.0,Male,-570.0,-511.0,Limb/Spinal,Short,0-39,False,36.0


---
# 9. Preprocessing Riluzole
---

In [63]:
df = utils_preprocessing.preprocess_riluzole(df_to_process=df, data_dir=data_dir)

df.head()

,subject_id,Age,Sex,Symptoms_Onset_Delta,Diagnosis_Delta,Site_Onset,Diagnosis_Delay,Age_at_Onset,Event_Dead,Event_Dead_Time_from_Onset,Riluzole
1,329,38.0,Female,-1706.0,-1068.0,Limb/Spinal,Long,0-39,False,65.0,False
2,348,52.0,Female,-501.0,-302.0,Other,Short,50-59,False,29.0,True
5,533,65.0,Female,-1023.0,-44.0,Bulbar,Long,60-69,False,46.0,False
6,586,63.0,Male,-715.0,-507.0,Bulbar,Short,60-69,False,23.0,False
8,708,46.0,Male,-1070.0,-986.0,Bulbar,Short,40-49,True,43.0,False


### ---
# 10. Preprocessing El Escorial
---

### _NOTE: El-Escorial has 36% of missing values, reducing the complete-cases from 5,271 to 3,380_

In [75]:
df = utils_preprocessing.preprocess_el_escorial(df_to_process=df, data_dir=data_dir)

display(df.head())

utils.show_columns_stats(df, columns=['El_Escorial'])

,subject_id,Age,Sex,Symptoms_Onset_Delta,Diagnosis_Delta,Site_Onset,Diagnosis_Delay,Age_at_Onset,Event_Dead,Event_Dead_Time_from_Onset,Riluzole,El_Escorial
1,329,38.0,Female,-1706.0,-1068.0,Limb/Spinal,Long,0-39,False,65.0,False,NaN
2,348,52.0,Female,-501.0,-302.0,Other,Short,50-59,False,29.0,True,Probable Laboratory Supported
5,533,65.0,Female,-1023.0,-44.0,Bulbar,Long,60-69,False,46.0,False,Probable Laboratory Supported
6,586,63.0,Male,-715.0,-507.0,Bulbar,Short,60-69,False,23.0,False,Probable Laboratory Supported
8,708,46.0,Male,-1070.0,-986.0,Bulbar,Short,40-49,True,43.0,False,NaN


El_Escorial =  3380 rows (64.12%)  1891 with NaN (35.88%) Uniques=     5 


---
# 11. Save the pre-processed data to CSV file
---

In [76]:
dir_dest = os.path.abspath('../03_preprocessed_data/')
csv_dest = f'{dir_dest}/patient.csv'
utils.save_to_csv(df=df, csv_file=csv_dest)


5271 samples were saved



---
---
---
---
# O T H E R S
---
---
---
---

---
# ?. Preprocessing ALS History
---

## Read ALS History data file

In [65]:
df_bkp = df.copy()

In [66]:
utils.show_columns_stats(df)

subject_id................ =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=  5271 
Age....................... =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=   189 
Sex....................... =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=     2 
Symptoms_Onset_Delta...... =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=  1319 
Diagnosis_Delta........... =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=   962 
Site_Onset................ =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=     4 
Diagnosis_Delay........... =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=     3 
Age_at_Onset.............. =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=     5 
Event_Dead................ =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=     2 
Event_Dead_Time_from_Onset =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=   111 
Riluzole.................. =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=     2 


In [51]:
df = df_bkp.copy()

In [68]:
utils.show_columns_stats(df_temp)

subject_id................ =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=  5271 
Age....................... =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=   189 
Sex....................... =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=     2 
Symptoms_Onset_Delta...... =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=  1319 
Diagnosis_Delta........... =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=   962 
Site_Onset................ =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=     4 
Diagnosis_Delay........... =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=     3 
Age_at_Onset.............. =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=     5 
Event_Dead................ =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=     2 
Event_Dead_Time_from_Onset =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=   111 
Riluzole.................. =  5271 rows (100.0%)     0 with NaN (  0.0%) Uniques=     2 
El_Escorial..........

---
# ?. Save the pre-processed data to CSV file
---

In [9]:
dir_dest = os.path.abspath('../03_preprocessed_data/')
csv_dest = f'{dir_dest}/patient.csv'
utils.save_to_csv(df=df, csv_file=csv_dest)


9598 samples were saved
